Workflow:
    ---------
    1. Extract text from the resume PDF.
    2. Split the resume into logical sections.
    3. Randomly select sections for enhancement.
    4. Use an the openAI API key to enhance selected sections based on the job description.
    5. Reconstruct the resume with enhanced sections.
    6. Save the final resume text as a PDF!


In [ ]:
!pip install pdfplumber
!pip install openai
!pip install fpdf

In [ ]:
!apt-get update && apt-get install -y fonts-dejavu


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import pdfplumber
import random
import openai
from fpdf import FPDF
import re
import os
import random
from getpass import getpass



Step 1: Job Description

In [ ]:
# Define job descriptions
job_description = [
    """Job Description 1:.NET Application Developer
•	Remote
•	Full Time
•	Mid Level
We are looking for a REMOTE .NET Application Developer to support our client based in Des Moines, Iowa
US BASED CANDIDATES ONLY.
This is a fully remote position.
Who we are
Founded in 2009 and headquartered in beautiful Miami, FL, TECKpert is a tech consulting and staff augmentation firm.  At TECKpert, we offer a contingent workforce built for any size digital transformation project. Experts in design, development, IT, analytics and marketing, provide innovative digital solutions to achieve success in our new economy. Our leaders identify the technical talent best suited to bolster our client’s capabilities, across all industries, including, healthcare, government, finance, legal, real estate, and startups.
The project
TECKpert seeks to hire a .NET Application Developer to support our client, a government agency based in Des Moines, IA.
Job duties include, but are not limited to:
•	Working with team on updating application requirements documentation; architecting and implementing improvements to data collection, validation, and reporting features across the application suite
•	Rebuild legacy code in C#/.NET
•	Work on education mission-focused solutions supporting educational institutions and student wellbeing
•	Work with the latest .Net technologies and tools
•	Work with the project team to analyze and document requirements; and help to refine complex data validations using MS SQL
•	Full stack web development with hands on experience with back-end tools
•	Independent work as well as collaborative work with the technical team and the close- knit team of business users/data analysts.
•	20% Business analysis meetings with stakeholders
•	30% Independent development work
•	30% Data refinement/SQL queries
•	20% Testing
This opportunity is for a full-time, one year contract position with possible extensions with pay commensurate with experience up to $47.35/hour or $98,488 per year. Medical, dental, vision and life insurance available after 30 days of hire.
Qualifications you need
A successful possesses or provides the following:
•	Bachelor’s degree in Computer Science or related field or relevant
work experience
•	C# development Required 4 Years
•	Experience in .Net projects Required 4 Years
•	Experience with MSSQL Stored Procedures, SSIS Packages Required 4 Years
•	Experience in asp.net/MVC web development using C# Required 4 Years
•	Experince working with MSSQL Server Required 4 Years
•	Hands on experience with VB.NET Required 4 Years
•	Hands on experience with JavaScript and jQuery Required 4 Years
•	Experience working with stakeholders in a business analysis capacity
•	Experience with Visual Studio 2017, Visual Studio 2022, MS SQl Server
•	Experience with Linq,Html, CSS, Bootstrap, Bootswatch
Working with us
Working with TECKpert means more options. As new opportunities arise, you tell us what you think is a good fit for you. What industries interest you most? Do you prefer an on-location, 9-5? Or would you want a flexible schedule and remote work? We proudly offer a wide variety of roles. Many of our TECKperts enjoy coworking and skills training coupled with the stability of full-time employment. We believe TECKpert gives today's digital professionals an agile path to start and advance their career. All of our opportunities require at least 20 hours per week and can be one to twelve months in length. Choose the opportunity that matches your interest and desired cadence.

 """]


OpenAI key

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")


🔑 Enter your OpenAI API key: ··········


In [ ]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

Step 2: define path

TODO: Mount the drive

In [ ]:
resume_pdf_path = "59_resume_job1_authentic_0.pdf"

In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

# extract_text_from_pdf(resume_pdf_path)

In [ ]:
# def split_into_sections(text):
#     # Use common resume section headers to split the text
#     section_patterns = [
#     r"(Experience|Work Experience|Professional Experience|Employment History|Career Summary)",
#     r"(Education|Academic Background)",
#     r"(Skills|Technical Skills|Core Competencies|Technical Proficiencies)",
#     r"(Projects|Personal Projects|Relevant Projects)",
#     r"(Certifications|Licenses|Licensure)",
#     r"(Summary|Professional Summary|Profile|Objective)",
#     r"(Achievements|Accomplishments|Awards|Honors)",
#     r"(Personal Information|Contact Information|Details)",
#     r"(Publications|Research)",
#     r"(Volunteer Experience|Volunteering|Community Service)",
#     r"(Languages|Language Skills)",
#     r"(Interests|Hobbies)"
#     r"(References)"
#     ]
#     sections = {}
#     current_section = "Header"
#     sections[current_section] = ""

#     for line in text.split("\n"):
#         line = line.strip()
#         if not line:
#             continue
#         matched = False
#         for pattern in section_patterns:
#             if re.search(pattern, line, re.IGNORECASE):
#                 current_section = line
#                 sections[current_section] = ""
#                 matched = True
#                 break
#         if not matched:
#             sections[current_section] += line + "\n"

#     return sections

#split_into_sections(extract_text_from_pdf(resume_pdf_path))


In [ ]:
def split_into_sections(text):
    section_patterns = [
        r"(Experience|Work Experience|Professional Experience|Employment History|Career Summary)",
        r"(Education|Academic Background)",
        r"(Skills|Technical Skills|Core Competencies|Technical Proficiencies)",
        r"(Projects|Personal Projects|Relevant Projects)",
        r"(Certifications|Licenses|Licensure)",
        r"(Summary|Professional Summary|Profile|Objective)",
        r"(Achievements|Accomplishments|Awards|Honors)",
        r"(Personal Information|Contact Information|Details)",
        r"(Publications|Research)",
        r"(Volunteer Experience|Volunteering|Community Service)",
        r"(Languages|Language Skills)",
        r"(Interests|Hobbies)",
        r"(References)"
    ]

    sections = {}
    current_section = "Header"
    sections[current_section] = ""

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue

        matched = False

        # Check against known section patterns
        for pattern in section_patterns:
            if re.search(pattern, line, re.IGNORECASE):
                current_section = line
                sections[current_section] = ""
                matched = True
                break

        # NEW: Check if line looks like an ALL CAPS section heading
        if not matched and line.isupper() and 1 <= len(line.split()) <= 4:
            current_section = line
            sections[current_section] = ""
            matched = True

        # If no new section found, append to current
        if not matched:
            sections[current_section] += line + "\n"

    return sections


In [ ]:
# sections = split_into_sections(extract_text_from_pdf(resume_pdf_path))

# # Print the number of sections and their names
# print(f"Total Sections Extracted: {len(sections)}")
# print("Extracted Sections:")
# for section in sections.keys():
#     print(f"- {section}")


In [ ]:
# # Function to enhance selected sections
# def enhance_selected_sections(selected_sections, job_description):
#     enhanced_sections = {}
#     for section, text in selected_sections.items():
#         enhanced_sections[section] = enhance_section(text, job_description)
#     return enhanced_sections


In [ ]:
# Function to randomly select sections
def select_random_sections(sections):
    section_keys = list(sections.keys())
    num_sections = random.choice([1, 2])  # Choose 1, 2, or all sections randomly
    selected_keys = random.sample(section_keys, num_sections)  # Select random sections
    selected_sections = {key: sections[key] for key in selected_keys}

    # Print selected sections
    print(f" Selected Sections for Enhancement ({num_sections} total):")
    for section in selected_keys:
        print(f"- {section}")

    return selected_sections
# select_random_sections(split_into_sections(extract_text_from_pdf(resume_pdf_path)))


In [ ]:
### SAME FUNCTION AS ABOVE BUT SETS MAX_SECTIONS TO 2
# def select_random_sections(sections, max_sections=2):
#     section_keys = list(sections.keys())
#     num_sections = min(max_sections, len(section_keys))  # cap to available sections
#     selected_keys = random.sample(section_keys, num_sections)
#     selected_sections = {key: sections[key] for key in selected_keys}

#     print(f"\n Selected Sections for Enhancement ({num_sections} total):")
#     for section in selected_keys:
#         print(f"- {section}")

#     return selected_sections

In [ ]:
def enhance_section(section_text, job_description):
    prompt = f"""
    I am applying for the following job:

    {job_description}

    Here is a section of my resume:
    "{section_text}"

    Please enhance and optimize this section to make it more compelling and relevant to this job. Keep the improved version professional and concise.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

# enhance_section(extract_text_from_pdf(resume_pdf_path), job_description)

In [ ]:
### ENHANCES SECTIONS AND AT THE SAME TIME ADDS PLACEHOLDERS
# def enhance_section(section_text, job_description):
#     prompt = f"""
#     I am applying for the following job:

#     {job_description}

#     Here is a section of my resume:
#     "{section_text}"

#     Please perform the following enhancements and anonymizations:
#     1. Enhance and optimize the section to make it more compelling, clear, and relevant to this job.
#     2. Maintain a professional and concise tone.
#     3. Anonymize the content by replacing any personal information with placeholders, including:
#        - Full names → <NAME>
#        - Emails → <EMAIL>
#        - Phone numbers → <PHONE>
#        - Companies → <COMPANY>
#        - Universities → <INSTITUTION>
#        - Addresses → <ADDRESS>
#        - URLs/LinkedIn → <LINK>

#     Output only the enhanced and anonymized version.
#     """

#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[{"role": "user", "content": prompt}]
#     )

#     return response.choices[0].message.content


In [ ]:
# Function to enhance selected sections
def enhance_selected_sections(selected_sections, job_description):
    enhanced_sections = {}
    print("\n🔹 Enhancing the following sections:")

    for section, text in selected_sections.items():
        print(f"✨ Enhancing: {section}")
        enhanced_sections[section] = enhance_section(text, job_description)

    return enhanced_sections

# enhance_selected_sections(select_random_sections(split_into_sections(extract_text_from_pdf(resume_pdf_path))), job_description)


In [ ]:
### ONLY FOR DEMONSTRATION PURPOSES
# def show_before_after(selected_sections, enhanced_sections):
#     print("\n🔹 Comparison: Before and After Enhancement 🔹\n")

#     for section in selected_sections.keys():
#         print(f"📝 **Section: {section}**\n")

#         print("🔹 **Before Enhancement:**")
#         print(selected_sections[section][:500] + "...")

#         if section in enhanced_sections:
#             print("\n✨ **After Enhancement:**")
#             print(enhanced_sections[section][:700] + "...")
#         else:
#             print("\n **After Enhancement:** Section not found!")

#         print("\n" + "="*80 + "\n")

In [ ]:
def reconstruct_resume(original_sections, enhanced_sections):
    """Combine enhanced and original sections into a clean resume string."""
    parts = []
    for section, text in original_sections.items():
        header = f"\n{section.strip()}\n" + "-" * len(section.strip())
        body = enhanced_sections.get(section, text).strip()
        parts.append(f"{header}\n{body}")
    return "\n\n".join(parts)


In [ ]:
from fpdf import FPDF
import os

def save_text_as_pdf(text, output_pdf_path):
    """Save the resume text to a clean, Unicode-compatible PDF."""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Download DejaVuSans if not present
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    if not os.path.exists(font_path):
        !apt-get update && apt-get install -y fonts-dejavu

    pdf.add_font("DejaVu", "", font_path, uni=True)
    pdf.set_font("DejaVu", "", 12)

    for paragraph in text.split("\n\n"):
        for line in paragraph.strip().split("\n"):
            pdf.multi_cell(0, 10, line.strip())
        pdf.ln(5)

    pdf.output(output_pdf_path)


In [ ]:
# # extract sections
# resume_text = extract_text_from_pdf(resume_pdf_path)

# # Split into sections
# resume_sections = split_into_sections(resume_text)

# # Randomly select sections
# selected_sections = select_random_sections(resume_sections)

# # Enhance sections with OpenAI
# enhanced_sections = enhance_selected_sections(selected_sections, job_description)

# # Reconstruct final resume
# final_resume_text = reconstruct_resume(resume_sections, enhanced_sections)

# # Save enhanced resume as PDF
# enhanced_resume_pdf_path = "enhanced_resume_update.pdf"
# save_text_as_pdf(final_resume_text, enhanced_resume_pdf_path)

# # Provide download link for the user in Colab
# # files.download(enhanced_resume_pdf_path)

# print(f"✅ Enhanced resume saved and ready for download: {enhanced_resume_pdf_path}")

In [ ]:
def process_and_enhance_resume(resume_pdf_path, job_description, output_pdf_path="enhanced_resume.pdf"):
    # Step 1: Extract text
    resume_text = extract_text_from_pdf(resume_pdf_path)

    # Step 2: Split into sections
    resume_sections = split_into_sections(resume_text)

    # Step 3: Select sections randomly
    selected_sections = select_random_sections(resume_sections)

    # Step 4: Enhance selected sections
    enhanced_sections = enhance_selected_sections(selected_sections, job_description)

    # Step 5: Reconstruct final resume
    final_resume_text = reconstruct_resume(resume_sections, enhanced_sections)

    # Step 6: Save to PDF
    save_text_as_pdf(final_resume_text, output_pdf_path)

    print(f"✅ Enhanced resume saved and ready for download: {output_pdf_path}")
    # return output_pdf_path


In [ ]:
from google.colab import files
files.download(enhanced_resume_pdf_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

# Set your directories
input_dir = "/content/drive/My Drive/CAPSTONE/authentic_resumes/filtered_2_page_resumes"
output_dir = "/content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# TO RUN ALL OF THEM, HOWEVER IT IS BETTER TO RUN THE TEST AND FINAL ONE BELOW TO ENSURE EVERYTHING IS RUNNING SMOOTHLY
# Loop through all PDF files in the input directory
# for filename in os.listdir(input_dir):
#     if filename.lower().endswith(".pdf"):
#         input_path = os.path.join(input_dir, filename)
#         output_filename = filename.replace(".pdf", "_enhanced.pdf")
#         output_path = os.path.join(output_dir, output_filename)

#         try:
#             print(f"\n🔄 Starting: {filename}")
#             process_and_enhance_resume(input_path, job_description, output_path)
#             print(f"✅ Finished: {output_filename}")
#         except Exception as e:
#             print(f"❌ Failed to process {filename}: {e}")



In [ ]:
pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".pdf")]

In [ ]:
# First 5 already processed
test_files = pdf_files[:5]

# Next 5 for further testing
more_test_files = pdf_files[5:10]

# Remaining files for full batch later
remaining_files = pdf_files[10:]



In [ ]:
print("TEST RUN (first 5 resumes):")
for filename in test_files:
    input_path = os.path.join(input_dir, filename)
    output_filename = filename.replace(".pdf", "_enhanced.pdf")
    output_path = os.path.join(output_dir, output_filename)

    try:
        print(f"\n🔄 Processing (TEST): {filename}")
        process_and_enhance_resume(input_path, job_description, output_path)
        print(f"✅ Test Complete: {output_filename}")
    except Exception as e:
        print(f"❌ Test failed for {filename}: {e}")


TEST RUN (first 5 resumes):

🔄 Processing (TEST): 118_resume_job4_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- digital transformation projects, excels at planning, managing, and executing multi-channel marketing and
- call of duty and utilizing excellent critical thinking and problem-solving skills to quickly evaluate issues and

🔹 Enhancing the following sections:
✨ Enhancing: digital transformation projects, excels at planning, managing, and executing multi-channel marketing and
✨ Enhancing: call of duty and utilizing excellent critical thinking and problem-solving skills to quickly evaluate issues and


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/118_resume_job4_authentic_0_enhanced.pdf
✅ Test Complete: 118_resume_job4_authentic_0_enhanced.pdf

🔄 Processing (TEST): 136_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- DeveloperandAdvancedGraduateCertificateinArtificialIntelligence.Fluentinmultipleprogramminglanguagesandframeworks.
- Summary

🔹 Enhancing the following sections:
✨ Enhancing: DeveloperandAdvancedGraduateCertificateinArtificialIntelligence.Fluentinmultipleprogramminglanguagesandframeworks.
✨ Enhancing: Summary


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/136_resume_job3_authentic_0_enhanced.pdf
✅ Test Complete: 136_resume_job3_authentic_0_enhanced.pdf

🔄 Processing (TEST): 78_resume_job4_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- CERTIFICATIONS AND TRAININGS
- EDUCATION

🔹 Enhancing the following sections:
✨ Enhancing: CERTIFICATIONS AND TRAININGS
✨ Enhancing: EDUCATION


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/78_resume_job4_authentic_0_enhanced.pdf
✅ Test Complete: 78_resume_job4_authentic_0_enhanced.pdf

🔄 Processing (TEST): 121_resume_job4_authentic_0.pdf


 Selected Sections for Enhancement (3 total):
- & 2016, and Mac OS/ iOS system experience.
- Header
- Information Technology professional with over a decade of experience in project management and customer

🔹 Enhancing the following sections:
✨ Enhancing: & 2016, and Mac OS/ iOS system experience.
✨ Enhancing: Header
✨ Enhancing: Information Technology professional with over a decade of experience in project management and customer
✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/121_resume_job4_authentic_0_enhanced.pdf
✅ Test Complete: 121_resume_job4_authentic_0_enhanced.pdf

🔄 Processing (TEST): 79_resume_job4_authentic_0.pdf
 Selected Sections for Enhancement (1 total):
- • Collaborating with partners across various skillsets to ensure successful release

🔹 Enhancing the following sections:
✨ Enhancing: • Collaborating with partners across various skillsets to ensure successful release
✅ Enhanced res

In [ ]:
print("🚀 SECOND TEST RUN (resumes 6–10):")

for filename in more_test_files:
    input_path = os.path.join(input_dir, filename)
    output_filename = filename.replace(".pdf", "_enhanced.pdf")
    output_path = os.path.join(output_dir, output_filename)

    try:
        print(f"\n🔄 Processing (TEST 2): {filename}")
        process_and_enhance_resume(input_path, job_description, output_path)
        print(f"✅ Finished: {output_filename}")
    except Exception as e:
        print(f"❌ Failed to process {filename}: {e}")


🚀 SECOND TEST RUN (resumes 6–10):

🔄 Processing (TEST 2): 59_resume_job4_authentic_0.pdf


 Selected Sections for Enhancement (3 total):
- Work Experience:
- Computer Skills:
- Education:

🔹 Enhancing the following sections:
✨ Enhancing: Work Experience:
✨ Enhancing: Computer Skills:
✨ Enhancing: Education:


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/59_resume_job4_authentic_0_enhanced.pdf
✅ Finished: 59_resume_job4_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 143_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- more seamless user experience.
- Work Experience

🔹 Enhancing the following sections:
✨ Enhancing: more seamless user experience.
✨ Enhancing: Work Experience


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/143_resume_job3_authentic_0_enhanced.pdf
✅ Finished: 143_resume_job3_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 84_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (3 total):
- EDUCATION
- SOFT SKILLS
- Header

🔹 Enhancing the following sections:
✨ Enhancing: EDUCATION
✨ Enhancing: SOFT SKILLS
✨ Enhancing: Header


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/84_resume_job3_authentic_0_enhanced.pdf
✅ Finished: 84_resume_job3_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 107_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (1 total):
- • Multi-industry experience beyond just IT, including electrical, HVAC, access control, safety

🔹 Enhancing the following sections:
✨ Enhancing: • Multi-industry experience beyond just IT, including electrical, HVAC, access control, safety


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/107_resume_job3_authentic_0_enhanced.pdf
✅ Finished: 107_resume_job3_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 56_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (3 total):
- PROFESSIONAL SUMMARY
- Systems Administrator and Navy veteran with 9 years of collective experience conducting network and systems
- JACOB CORDANO

🔹 Enhancing the following sections:
✨ Enhancing: PROFESSIONAL SUMMARY
✨ Enhancing: Systems Administrator and Navy veteran with 9 years of collective experience conducting network and systems
✨ Enhancing: JACOB CORDANO
✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/56_resume_job3_authentic_0_enhanced.pdf
✅ Finished: 56_resume_job3_authentic_0_enhanced.pdf


In [ ]:
print("🚀 REMAINING TEST RUN (resumes 11-141):")

for filename in remaining_files:
    input_path = os.path.join(input_dir, filename)
    output_filename = filename.replace(".pdf", "_enhanced.pdf")
    output_path = os.path.join(output_dir, output_filename)

    try:
        print(f"\n🔄 Processing (TEST 2): {filename}")
        process_and_enhance_resume(input_path, job_description, output_path)
        print(f"✅ Finished: {output_filename}")
    except Exception as e:
        print(f"❌ Failed to process {filename}: {e}")

🚀 REMAINING TEST RUN (resumes 11-141):

🔄 Processing (TEST 2): 27_resume_job3_authentic_0.pdf


 Selected Sections for Enhancement (1 total):
- ➢ Managed numerous multi-tenant and single tenant migration projects, specializing in virtualization via Citrix and VMware as well

🔹 Enhancing the following sections:
✨ Enhancing: ➢ Managed numerous multi-tenant and single tenant migration projects, specializing in virtualization via Citrix and VMware as well


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/27_resume_job3_authentic_0_enhanced.pdf
✅ Finished: 27_resume_job3_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 51_resume_job4_authentic_0.pdf
 Selected Sections for Enhancement (1 total):
- Header

🔹 Enhancing the following sections:
✨ Enhancing: Header


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/51_resume_job4_authentic_0_enhanced.pdf
✅ Finished: 51_resume_job4_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 141_resume_job2_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- ○ Engineered a reusable input validation library using Java, promoting code reusability and enhancing collaboration across various internal projects.
- Results-driven Java Developer with a Master’s in Computer Software Engineering and extensive experience in full-stack development, backend engineering,

🔹 Enhancing the following sections:
✨ Enhancing: ○ Engineered a reusable input validation library using Java, promoting code reusability and enhancing collaboration across various internal projects.
✨ Enhancing: Results-driven Java Developer with a Master’s in Computer Software Engineering and extensive experience in full-stack development, backend engineering,
✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/141_resume_job2_authentic_0_enhanced.pdf
✅ Finished: 141_resume_job2_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 94_resume_job3_authentic_0.pdf
 S

...

 Selected Sections for Enhancement (1 total):
- skills.

🔹 Enhancing the following sections:
✨ Enhancing: skills.


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/81_resume_job5_authentic_0_enhanced.pdf
✅ Finished: 81_resume_job5_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 67_resume_job1_authentic_0.pdf


 Selected Sections for Enhancement (2 total):
- ● Great Plains ● Excellent analytical skills
- ● Engaged in a focus group to plan business future data systems goals and objectives for clients and internal use. In an acting role of leader

🔹 Enhancing the following sections:
✨ Enhancing: ● Great Plains ● Excellent analytical skills
✨ Enhancing: ● Engaged in a focus group to plan business future data systems goals and objectives for clients and internal use. In an acting role of leader


✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/67_resume_job1_authentic_0_enhanced.pdf
✅ Finished: 67_resume_job1_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 39_resume_job2_authentic_0.pdf


 Selected Sections for Enhancement (1 total):
- Programming Skills: HTML, CSS, JavaScript, C, C++, MySQL, SAP HANA SQL, Crystal

🔹 Enhancing the following sections:
✨ Enhancing: Programming Skills: HTML, CSS, JavaScript, C, C++, MySQL, SAP HANA SQL, Crystal
✅ Enhanced resume saved and ready for download: /content/drive/My Drive/CAPSTONE/authentic_resumes/enhanced_filtered_2_page_resumes/39_resume_job2_authentic_0_enhanced.pdf
✅ Finished: 39_resume_job2_authentic_0_enhanced.pdf

🔄 Processing (TEST 2): 122_resume_job1_authentic_0.pdf
 Selected Sections for Enhancement (2 total):
- Skills: Excellent oral and written communication skills, securities, finance.
- BENJAMIN YASHAR

🔹 Enhancing the following sections:
✨ Enhancing: Skills: Excellent oral and written communication skills, securities, finance.
✨ Enhancing: BENJAMIN YASHAR
❌ Failed to process 122_resume_job1_authentic_0.pdf: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing

In [ ]:
print("i hope it doesn't disconnect")

i hope it doesn't disconnect


In [ ]:
start_index = pdf_files.index("122_resume_job1_authentic_0.pdf")
print(start_index)

140


In [ ]:
resume_from_here = pdf_files[start_index:]


In [ ]:
print(f"🚀 RESUMING FROM: {resume_from_here[0]}")

for filename in resume_from_here:
    input_path = os.path.join(input_dir, filename)
    output_filename = filename.replace(".pdf", "_enhanced.pdf")
    output_path = os.path.join(output_dir, output_filename)

    try:
        print(f"\n🔄 Processing: {filename}")
        process_and_enhance_resume(input_path, job_description, output_path)
        print(f"✅ Finished: {output_filename}")
    except Exception as e:
        print(f"❌ Failed to process {filename}: {e}")


NameError: name 'resume_from_here' is not defined